# California Fire Information

In [1]:
# Import libaries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

Obtain current data from Cal-Fire website

In [2]:
# set url
base_url = 'https://www.fire.ca.gov/incidents/'
res = requests.get(base_url)
# print status code
res.status_code

200

In [3]:
# create soup from base url
soup = BeautifulSoup(res.content, 'lxml')

In [4]:
# examine soup object
# print(soup.prettify())

In [5]:
# create table 
table = soup.find('div',{'class':'featured-incidents'})

In [6]:
# examine table
# print(table.prettify())

In [7]:
# cell used to isolate info
for i in table.find_all('div',{'class':'featured-incident'}):
    print(i.find('div',{'class':'incident-description'}).find('strong').text)


7:12am
10/23/2019
10/27/2019
10/27/2019
7:30am
7:31am
6:47am
10/27/2019
10/27/2019
10/27/2019
10/24/2019
10/24/2019
10/22/2019
10/21/2019
10/14/2019
10/04/2019


In [8]:
# loop through list to obtain desired info and create dataframe
fires = []

for i in table.find_all('div',{'class':'featured-incident'}):
    fire = {}
    
    fire['name'] = i.find('div',{'class':'incident-label'}).text
    fire['lat'] = i.attrs['data-lat']
    fire['lng'] = i.attrs['data-long']
    fire['%_contained'] = i.find('div',{'class':'incident-containment'}).attrs['style'].replace('width:', '').replace(';', '')
    fire['acres'] = i.find('span',{'class':'incident-acres'}).text.replace('acres', '')
    fire['current_as_of'] = i.find('div',{'class':'incident-description'}).find('strong').text
    fires.append(fire)

df = pd.DataFrame(fires)
df    

,%_contained,acres,current_as_of,lat,lng,name
0,5%,66231,7:12am,38.792458,-122.780053,Kincade Fire
1,97%,8799,10/23/2019,34.32988,-118.48161,Saddle Ridge Fire
2,70%,4615,10/27/2019,34.472778,-118.368056,Tick Fire
3,20%,350,10/27/2019,39.22431,-123.12887,Burris Fire
4,90%,150,7:30am,38.03545,-122.2241,Sky Fire
5,98%,97,7:31am,33.055124,-116.742508,Sawday Fire
6,%,Not a CAL FIRE Incident,6:47am,34.098191,-118.478717,Getty Fire
7,%,Not a CAL FIRE Incident,10/27/2019,37.97911,-122.11092,Forest Fire
8,%,Not a CAL FIRE Incident,10/27/2019,38.07205,-122.22463,Glencove Fire
9,%,Not a CAL FIRE Incident,10/27/2019,37.99136,-121.64118,Cypress Complex


In [9]:
df.dtypes

%_contained      object
acres            object
current_as_of    object
lat              object
lng              object
name             object
dtype: object

In [10]:
# cast lat and lng to float
df[['lat', 'lng']] = df[['lat', 'lng']].astype(float)
df.dtypes

%_contained       object
acres             object
current_as_of     object
lat              float64
lng              float64
name              object
dtype: object

## Display fires on map

In [11]:
import folium
from folium.features import DivIcon

In [12]:
from PIL import Image # converting images into arrays

In [13]:
# California latitude and longitude values
latitude = 36.7783
longitude = -119.4179

# create map and display it
cali_map = folium.Map(location=[latitude, longitude], zoom_start=6, tiles='Stamen Terrain')

# add markers for fires
labels = list(df.name)

latitudes = list(df.lat)
longitudes = list(df.lng)

# Add fire as an icon
icon_url = 'fire-icon.png'

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label, 
                  icon=folium.features.CustomIcon(icon_url, icon_size=(35,35))).add_to(cali_map) 

# add dateframe to map    
folium.map.Marker(
    [40.717893, -128.565340],
    icon=DivIcon(
        icon_anchor=(0,0),
        icon_size=(1,1),
        
        html=df[['name', 'acres', '%_contained']].head(10).to_html(index=False),
        )
    ).add_to(cali_map)


# display the map of California
cali_map

In [14]:
# save as html file
cali_map.save('index.html')

# Plot Numeric Data

In [49]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Bar(x=df['name'][:5], y=df['acres'][:5]))
fig.show()

In [47]:
fig = go.Figure(data=go.Bar(x=df['name'][:5], y=df['%_numeric'][:5]))
fig.show()

In [73]:
x=df['name'][:5] 
y=df['%_numeric'][:5]
y2=df['acres'][:5]

In [88]:
fig1 = go.Figure(data=go.Bar(x=df['name'][:5], y=df['%_numeric'][:5],
                           text=y,
    textposition = 'auto',
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6))

fig1.update_layout(
    title=go.layout.Title(
        text="Percent Contained",
        xref="paper",
        x=0
    ))

fig1.show()

In [ ]:
pio.write_html(fig1, file='percent_contained.html', auto_open=True)

In [89]:
fig2 = go.Figure(data=go.Bar(x=df['name'][:5], y=df['acres'][:5],
                            text=y2,
    textposition = 'auto',
    marker=dict(
        color='rgb(58,200,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
         ),
    opacity=0.6))

fig2.update_layout(
    title=go.layout.Title(
        text="Acres Affected",
        xref="paper",
        x=0
    ))

fig2.show()       

In [ ]:
pio.write_html(fig1, file='acres_affected.html', auto_open=True)

In [87]:
import plotly.io as pio

In [115]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.7],
                   subplot_titles=("Percent Contained", "Acres Affected"))

fig.add_trace(go.Bar(x=df['name'][:5], y=df['%_numeric'][:5],
                                    text=y, textposition = 'auto',
                                    marker=dict(
                                        color='rgb(158,202,225)',
                                        line=dict(
                                            color='rgb(8,48,107)',
                                            width=1.5)), opacity=0.6), row=1, col=1)

fig.add_trace(go.Bar(x=df['name'][:5], y=df['acres'][:5],
                            text=y2,
    textposition = 'auto',
    marker=dict(
        color='rgb(58,200,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
         ),
    opacity=0.6), row=1, col=2)
             
fig.update_layout(showlegend=False, title_text="California Wildfires as of October 28")            

fig.show()

In [116]:
pio.write_html(fig, file='graphs.html', auto_open=True)